In [1]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

In [2]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

In [7]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images('train'))
data = []
labels = []
# loop over the image paths
for imagePath in imagePaths:
    # extract the class label from the filename
    print(imagePath)
    label = imagePath.split(os.path.sep)[-2]
    # load the input image (224x224) and preprocess it
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)
# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

[INFO] loading images...
train\without_mask\0.jpg
train\without_mask\1.jpg
train\without_mask\10.jpg
train\without_mask\100.jpg
train\without_mask\101.jpg
train\without_mask\102.jpg
train\without_mask\104.jpg
train\without_mask\105.jpg
train\without_mask\106.jpg
train\without_mask\107.jpg
train\without_mask\108.jpg
train\without_mask\109.jpg
train\without_mask\11.jpg
train\without_mask\110.jpg
train\without_mask\111.jpg
train\without_mask\112.jpg
train\without_mask\114.jpg
train\without_mask\115.jpg
train\without_mask\116.jpg
train\without_mask\117.jpg
train\without_mask\11707.jpg
train\without_mask\118.jpg
train\without_mask\119.jpg
train\without_mask\12.jpg
train\without_mask\120.jpg
train\without_mask\122.jpg
train\without_mask\123.jpg
train\without_mask\124.jpg
train\without_mask\125.jpg
train\without_mask\127.jpg
train\without_mask\128.jpg
train\without_mask\129.jpg
train\without_mask\13.jpg
train\without_mask\130.jpg
train\without_mask\131.jpg
train\without_mask\132.jpg
train\wit

train\without_mask\421.jpg
train\without_mask\422.jpg
train\without_mask\423.jpg
train\without_mask\424.jpg
train\without_mask\425.jpg
train\without_mask\426.jpg
train\without_mask\427.jpg
train\without_mask\428.jpg
train\without_mask\43.jpg
train\without_mask\430.jpg
train\without_mask\431.jpg
train\without_mask\432.jpg
train\without_mask\434.jpg
train\without_mask\435.jpg
train\without_mask\436.jpg
train\without_mask\437.jpg
train\without_mask\438.jpg
train\without_mask\439.jpg
train\without_mask\44.jpg
train\without_mask\440.jpg
train\without_mask\441.jpg
train\without_mask\442.jpg
train\without_mask\443.jpg
train\without_mask\444.jpg
train\without_mask\445.jpg
train\without_mask\446.jpg
train\without_mask\447.jpg
train\without_mask\448.jpg
train\without_mask\449.jpg
train\without_mask\44a0110a-fb6b-4e24-ba06-8c0b91fc06db.jpg
train\without_mask\45.jpg
train\without_mask\450.jpg
train\without_mask\451.jpg
train\without_mask\452.jpg
train\without_mask\453.jpg
train\without_mask\454.jp

train\without_mask\augmented_image_220.jpg
train\without_mask\augmented_image_221.jpg
train\without_mask\augmented_image_222.jpg
train\without_mask\augmented_image_223.jpg
train\without_mask\augmented_image_224.jpg
train\without_mask\augmented_image_225.jpg
train\without_mask\augmented_image_226.jpg
train\without_mask\augmented_image_227.jpg
train\without_mask\augmented_image_229.jpg
train\without_mask\augmented_image_23.jpg
train\without_mask\augmented_image_230.jpg
train\without_mask\augmented_image_231.jpg
train\without_mask\augmented_image_232.jpg
train\without_mask\augmented_image_233.jpg
train\without_mask\augmented_image_234.jpg
train\without_mask\augmented_image_235.jpg
train\without_mask\augmented_image_236.jpg
train\without_mask\augmented_image_237.jpg
train\without_mask\augmented_image_238.jpg
train\without_mask\augmented_image_239.jpg
train\without_mask\augmented_image_240.jpg
train\without_mask\augmented_image_241.jpg
train\without_mask\augmented_image_243.jpg
train\withou

train\without_mask\offset-870274.jpg
train\without_mask\photo-1493106819501-66d381c466f1.jpeg
train\without_mask\pm.jpg
train\without_mask\pm1.jpg
train\without_mask\pm2.jpg
train\without_mask\pm3.jpg
train\without_mask\short-hairstyles-for-indian-faces-406016-50-indian-hairstyle.jpg
train\without_mask\short-hairstyles-for-indian-faces-406016-indian-hairstyle-me.jpg
train\without_mask\top-50-best-faces.jpg
train\without_mask\UTf8bVN3.jpeg
train\without_mask\want-to-see-more-indian-faces-in-us-government-raj-mukherji.jpg
train\with_mask\-110603108-gettyimages-533567012.jpg
train\with_mask\-110695084-mask2.jpg
train\with_mask\0-with-mask.jpg
train\with_mask\0000305-62360-halyard-health-pfr-p3-respirator-face-mask-ffp.jpeg
train\with_mask\0000444-48296-halyard-health-surgical-mask-teddy-bear-print.jpeg
train\with_mask\007563.jpg
train\with_mask\0779db20580fc7b054ad6b1b7bd9fb35.jpeg
train\with_mask\1-respirator-mask.jpg
train\with_mask\1-with-mask.jpg
train\with_mask\10-with-mask.jpg
train

train\with_mask\330-with-mask.jpg
train\with_mask\331-with-mask.jpg
train\with_mask\332-with-mask.jpg
train\with_mask\333-with-mask.jpg
train\with_mask\334-with-mask.jpg
train\with_mask\335-with-mask.jpg
train\with_mask\336-with-mask.jpg
train\with_mask\338-with-mask.jpg
train\with_mask\339-with-mask.jpg
train\with_mask\34-with-mask.jpg
train\with_mask\340-with-mask.jpg
train\with_mask\342-with-mask.jpg
train\with_mask\343-with-mask.jpg
train\with_mask\344-with-mask.jpg
train\with_mask\345-with-mask.jpg
train\with_mask\346-with-mask.jpg
train\with_mask\347-with-mask.jpg
train\with_mask\348-with-mask.jpg
train\with_mask\349-with-mask.jpg
train\with_mask\35-with-mask.jpg
train\with_mask\351-with-mask.jpg
train\with_mask\353-with-mask.jpg
train\with_mask\355-with-mask.jpg
train\with_mask\356-with-mask.jpg
train\with_mask\357-with-mask.jpg
train\with_mask\358-with-mask.jpg
train\with_mask\359-with-mask.jpg
train\with_mask\36-with-mask.jpg
train\with_mask\362-with-mask.jpg
train\with_mask\3

train\with_mask\augmented_image_145.jpg
train\with_mask\augmented_image_146.jpg
train\with_mask\augmented_image_147.jpg
train\with_mask\augmented_image_148.jpg
train\with_mask\augmented_image_149.jpg
train\with_mask\augmented_image_15.jpg
train\with_mask\augmented_image_150.jpg
train\with_mask\augmented_image_151.jpg
train\with_mask\augmented_image_152.jpg
train\with_mask\augmented_image_153.jpg
train\with_mask\augmented_image_154.jpg
train\with_mask\augmented_image_155.jpg
train\with_mask\augmented_image_156.jpg
train\with_mask\augmented_image_157.jpg
train\with_mask\augmented_image_158.jpg
train\with_mask\augmented_image_159.jpg
train\with_mask\augmented_image_16.jpg
train\with_mask\augmented_image_160.jpg
train\with_mask\augmented_image_161.jpg
train\with_mask\augmented_image_162.jpg
train\with_mask\augmented_image_163.jpg
train\with_mask\augmented_image_164.jpg
train\with_mask\augmented_image_165.jpg
train\with_mask\augmented_image_166.jpg
train\with_mask\augmented_image_167.jpg
tr

train\with_mask\augmented_image_93.jpg
train\with_mask\augmented_image_94.jpg
train\with_mask\augmented_image_96.jpg
train\with_mask\augmented_image_97.jpg
train\with_mask\augmented_image_98.jpg
train\with_mask\augmented_image_99.jpg
train\with_mask\depositphotos-8449546-stock-photo-flu-illness-child-boy-in.jpg
train\with_mask\Disposable-Ear-Loop-Face-Mask-Flu-Masks-Nose-Allergy-Dust-Ma.jpg
train\with_mask\ds1917.jpg
train\with_mask\FaceMasks--banner--GettyImages.jpg
train\with_mask\Female-Face-Mask-732x549-thumbnail-1-732x549.jpg
train\with_mask\Flu-Face-Mask-1.jpg
train\with_mask\flu-surgical-face-mask.jpg
train\with_mask\FLUIDSHIELD-N95-RESPIRATOR-REGULAR-PFR95-170-24511871.jpg
train\with_mask\flumask-feature-470x313.jpg
train\with_mask\Fun-flu-prevention.jpg
train\with_mask\gettyimages-1181396760.jpg
train\with_mask\gettyimages-713771083-1024x1024.jpg
train\with_mask\gettyimages-77932927-1024x1024.jpg
train\with_mask\grid-wide-surgical-mask-at752007.jpg
train\with_mask\high-filtrat

Grabbing all of the imagePaths in the dataset (Line 44)
Initializing data and labels lists (Lines 45 and 46)
Looping over the imagePaths and loading + pre-processing images (Lines 49-60). Pre-processing steps include resizing to 224×224 pixels, conversion to array format, and scaling the pixel intensities in the input image to the range [-1, 1] (via the preprocess_input convenience function)

In [8]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [14]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

Load MobileNet with pre-trained ImageNet weights, leaving off head of network (Lines 88 and 89)
Construct a new FC head, and append it to the base in place of the old head (Lines 93-102)
Freeze the base layers of the network (Lines 106 and 107). The weights of these base layers will not be updated during the process of backpropagation, whereas the head layer weights will be tuned.

In [15]:
#compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=64),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=1)

[INFO] compiling model...
[INFO] training head...
19/19 [==============================] - 363s 19s/step - loss: 0.8397 - acc: 0.5577 - val_loss: 0.3013 - val_acc: 0.9238


In [19]:
model.save('face_mask_model.h5')